In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import warnings
import datetime as dt
from scipy.interpolate import CubicSpline
from matplotlib import gridspec

import importlib
import sklearn

import xgboost as xgb

# local imports
from panda_helpers import *

from makedf import *

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 4.

dosave = False
savedir = "/icarus/app/users/gputnam/evtsel-ana/plots_Jan9/"

HMASS = 260
labelM = "$M_S$ = %iMeV" % HMASS
label = labelM + "\n$\\theta_S = 5\\cdot10^{-5}$"

# AMASS = 400
# labelM = "$M_a$ = %i MeV" % AMASS
# label = labelM + "\n$f_a$ = \n$c_\\ell$ ="

In [ ]:
fnu = "/icarus/data/users/gputnam/MCP2022GB/nu/nuall_redo.df"

# fnu = "/icarus/data/users/gputnam/MCP2022P/mc/numi_2d_mc_evt.df"

In [ ]:
nu_evtdf = pd.read_hdf(fnu, key="evtdf")

In [ ]:
nu_evtdf

In [ ]:
nu_mcdf = pd.read_hdf(fnu, key="numcdf")

In [ ]:
nu_mcdf

In [ ]:
nu_mcdf.lmom

In [ ]:
nu_hdr = pd.read_hdf(fnu, key="hdr")

In [ ]:
nu_hdr

In [ ]:
fh = "/icarus/data/users/gputnam/MCP2022GB/higgs/MCP2021P-higgs-260.df"
# fh = "/icarus/data/users/gputnam/MCP2022P/mc/alp_M400.df"

# fh = "/icarus/data/users/gputnam/MCP2022P/mc/higgs_M240.df"

In [ ]:
h_evtdf = pd.read_hdf(fh, key="evtdf")

In [ ]:
h_evtdf

In [ ]:
h_mcdf = pd.read_hdf(fh, key="mcdf")

In [ ]:
h_mcdf

In [ ]:
h_hdr = pd.read_hdf(fh, key="hdr")

In [ ]:
h_hdr

In [ ]:
fic = "/icarus/data/users/gputnam/MCP2022GB/cosmic/intc_5000.df"

In [ ]:
ic_evtdf = pd.read_hdf(fic, key="evtdf")

In [ ]:
ic_evtdf

In [ ]:
ic_hdr = pd.read_hdf(fic, key="hdr")

In [ ]:
ic_hdr

In [ ]:
nu_mcdf.index = nu_mcdf.index.set_levels(nu_mcdf.index.levels[0] + 1, level=0)

In [ ]:
def sample_concat(dfA, dfB, offset=1_000_000):
    dfB = dfB.copy()
    # dfB.index = dfB.index.set_levels(dfB.index.levels[0] + dfA.index.max()[0] + 1, level=0)
    dfB.index = dfB.index.set_levels(dfB.index.levels[0] + offset, level=0)
    return dfA.append(dfB)

In [ ]:
nu_pot = np.sum(nu_hdr.pot * nu_hdr.first_in_subrun)
h_pot = np.sum(h_hdr.pot * h_hdr.first_in_subrun)

In [ ]:
nu_hdr.pot

In [ ]:
nu_evt_per_spill = np.sum(nu_hdr.ngenevt*nu_hdr.first_in_subrun) / (np.sum(nu_hdr.pot*nu_hdr.first_in_subrun) / 6e13)
ic_pot = np.sum(ic_hdr.ngenevt * ic_hdr.first_in_subrun * 6e13) / (1 - nu_evt_per_spill)

In [ ]:
GOAL_POT = 2e20 
nu_evtdf["scale"] = GOAL_POT / nu_pot
h_evtdf["scale"] = GOAL_POT / h_pot
ic_evtdf["scale"] = GOAL_POT / ic_pot

nu_mcdf["scale"] = GOAL_POT / nu_pot
h_mcdf["scale"] = GOAL_POT / h_pot

POTSTR = "2e20 POT"

In [ ]:
ic_pot, nu_pot

In [ ]:
h_evtdf["higgs"] = True
nu_evtdf["higgs"] = False
ic_evtdf["higgs"] = False

In [ ]:
def decay_weight(in_dist, out_dist, mean_dist):
    return np.exp(-in_dist / mean_dist) - \
          np.exp(-out_dist / mean_dist)
    
def flux_weight(mixing):
    return mixing * mixing

def reweight_mixing(newmixing, start, enter, exit, mean_dist, oldmixing=1e-5):
    dist_in = dist(enter, start)
    dist_out = dist(exit, start)
    
    old_dcy = decay_weight(dist_in, dist_out, mean_dist)
    new_dcy = decay_weight(dist_in, dist_out, mean_dist / (newmixing**2 / oldmixing**2))
    
    old_flux = flux_weight(oldmixing)
    new_flux = flux_weight(newmixing)
    
    return (new_flux / old_flux) * (new_dcy / old_dcy)

In [ ]:
reweight_mixing

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 5, 21)

_ = plt.hist([var[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],
              var[when & (nu_mcdf.np == 0) & ((nu_mcdf.genie_mode != 3) | (nu_mcdf.iscc == 0))],
              var[when & (nu_mcdf.np > 0)],
             ],
              weights=[
                  nu_mcdf.scale[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],
                  nu_mcdf.scale[when & (nu_mcdf.np == 0) & ((nu_mcdf.genie_mode != 3) | (nu_mcdf.iscc == 0))],
                  nu_mcdf.scale[when & (nu_mcdf.np > 0)]
              ],
              bins=bins, label=["CC Coh", "No p", "With p"],
             stacked=True
            )
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy_splitnu.pdf")

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 10, 21)

_ = plt.hist([var[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],],
              weights=[nu_mcdf.scale[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],],
              bins=bins, label=["CC Coh"],
             stacked=True
            )
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy_cohnu.pdf")

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 5, 21)

_ = plt.hist(var[when], bins=bins, weights=nu_mcdf.scale[when])
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy.pdf")

In [ ]:
var = h_mcdf.E
when = SlcInFV(h_mcdf.position)
bins = np.linspace(0, 5, 21)
bins = np.linspace(0, 10, 21)

_ = plt.hist(var[when], bins=bins, weights=h_mcdf.scale[when])
plt.ylabel("Fiducial Scalars / %s" % POTSTR)
plt.xlabel("Scalar Energy [GeV]")
plt.tight_layout()
plt.text(0.6, 0.6, label, fontsize=18, transform=plt.gca().transAxes)
if dosave: plt.savefig(savedir + "scalar_energy_M%i.pdf" % HMASS)

In [ ]:
numu_cc = (np.abs(nu_mcdf.pdg) == 14) & nu_mcdf.iscc

mc_is_nu_coh = (nu_mcdf.genie_mode == 3) & numu_cc
mc_is_nu_coh.name = "$\\nu_\\mu$ CC COH"

mc_is_nu_pi = (nu_mcdf.npi >= 1) & (nu_mcdf.max_proton_ke < 0.02) & (nu_mcdf.npi0 == 0) & ~mc_is_nu_coh & numu_cc
mc_is_nu_pi.name = "$\\nu_\\mu$ CC n$\\pi$0p"
                                              
mc_is_nu_pip = (nu_mcdf.npi >= 1) & (nu_mcdf.max_proton_ke >= 0.02) & (nu_mcdf.npi0 == 0) & ~mc_is_nu_coh & numu_cc
mc_is_nu_pip.name = "$\\nu_\\mu$ CC n$\\pi$np"                                              

mc_is_nu_other = ~mc_is_nu_coh & ~mc_is_nu_pi & ~mc_is_nu_pip & numu_cc
mc_is_nu_other.name = "$\\nu_\\mu$ CC Other"

In [ ]:
mc_nucategories = [mc_is_nu_other, mc_is_nu_pip, mc_is_nu_pi, mc_is_nu_coh] #[mc_is_nu_pi, mc_is_nu_pip, mc_is_nu_coh, mc_is_nu_other]

In [ ]:
modes = [0, 1, 2, 3, 10]
names = ["QE", "RES", "DIS", "COH", "MEC"]

In [ ]:
var = nu_mcdf.E

when = SlcInFV(nu_mcdf.position)

bins = np.linspace(0, 10, 21)

N,_,_ = plt.hist([var[when & c] for c in mc_nucategories], 
         weights=[nu_mcdf.scale[when & c]/10 for c in mc_nucategories], bins=bins, 
         label=[c.name for c in mc_nucategories], 
        stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Neutrinos / %s" % POTSTR)
print(N.sum())
plt.tight_layout()
if dosave: plt.savefig(savedir + "all_nue_mccategories.pdf")

In [ ]:
var = nu_mcdf.lmom

when = SlcInFV(nu_mcdf.position)

bins = np.linspace(0, 5, 21)

N,_,_ = plt.hist([var[when & c] for c in mc_nucategories], 
         weights=[nu_mcdf.scale[when & c]/10 for c in mc_nucategories], bins=bins, 
         label=[c.name for c in mc_nucategories], 
        stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Muon Momentum [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Neutrinos / %s" % POTSTR)
print(N.sum())
plt.tight_layout()
if dosave: plt.savefig(savedir + "all_lmom_mccategories.pdf")

In [ ]:
var = nu_mcdf.E

when = SlcInFV(nu_mcdf.position) & numu_cc

bins = np.linspace(0, 10, 21)

# for c in nucategories:
_ = plt.hist([var[when & (nu_mcdf.genie_mode == m)] for m in modes], 
             weights=[nu_mcdf.scale[when & (nu_mcdf.genie_mode == m)]/10 for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Neutrinos / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "all_nue_intmodes.pdf")

In [ ]:
var = nu_mcdf.lmom

when = SlcInFV(nu_mcdf.position) & numu_cc

bins = np.linspace(0, 5, 21)

# for c in nucategories:
_ = plt.hist([var[when & (nu_mcdf.genie_mode == m)] for m in modes], 
             weights=[nu_mcdf.scale[when & (nu_mcdf.genie_mode == m)]/10 for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Muon Momentum [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Neutrinos / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "all_lmom_intmodes.pdf")

In [ ]:
evtdf = sample_concat(sample_concat(h_evtdf, nu_evtdf, offset=1), ic_evtdf)

In [ ]:
is_higgs = (evtdf.slc.tmatch.idx >= 0) & evtdf.higgs & (evtdf.slc.truth.npi == 0) &(evtdf.slc.truth.npi0 == 0)
is_higgs.name = "Scalar"
is_nu = (evtdf.slc.tmatch.idx >= 0) & ~evtdf.higgs
is_nu.name = "$\\nu$"
is_cosmic = evtdf.slc.tmatch.idx < 0
is_cosmic.name = "Cosmic"

In [ ]:
categories = [is_higgs, is_nu, is_cosmic]

In [ ]:
evt_numu_cc = (np.abs(evtdf.slc.truth.pdg) == 14) & evtdf.slc.truth.iscc

is_nu_coh = is_nu & (evtdf.slc.truth.genie_mode == 3) & (evtdf.slc.truth.iscc) & evt_numu_cc
is_nu_coh.name = "$\\nu_\\mu$ CC COH"

is_nu_pi = is_nu & (evtdf.slc.truth.npi >= 1) & (evtdf.slc.truth.max_proton_ke < 0.02) & (evtdf.slc.truth.npi0 == 0) & (evtdf.slc.truth.iscc) & evt_numu_cc & ~is_nu_coh
is_nu_pi.name = "$\\nu_\\mu$ CC n$\\pi$0p"
                                              
is_nu_pip = is_nu & (evtdf.slc.truth.npi >= 1) & (evtdf.slc.truth.max_proton_ke >= 0.02) & (evtdf.slc.truth.npi0 == 0) & (evtdf.slc.truth.iscc) & evt_numu_cc & ~is_nu_coh
is_nu_pip.name = "$\\nu_\\mu$ CC n$\\pi$np"                                              

is_nu_other = is_nu  & ~is_nu_coh & ~is_nu_pi & ~is_nu_pip
is_nu_other.name = "$\\nu$ Other"

In [ ]:
nucategories = [is_nu_other, is_nu_pip, is_nu_pi, is_nu_coh]

In [ ]:
scale = evtdf.scale.copy()
# scale[is_higgs] = evtdf.scale

In [ ]:
var = np.maximum(evtdf.trunk.len, evtdf.branch.len)

when = SlcInFV(evtdf.slc.vertex) & ~np.isnan(evtdf.slc.nu_score)

bins = np.linspace(0, 400, 21)

# for c in nucategories:
_ = plt.hist([var[when & c] for c in [is_cosmic] + nucategories], 
             weights=[scale[when & c]/10 for c in [is_cosmic] + nucategories], bins=bins, 
             label=[c.name for c in [is_cosmic] + nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Maximum Track Length [cm]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "allevt_trklen.pdf")

In [ ]:
evtdf.trunk.truth[is_higgs][:50]

In [ ]:
_ = plt.hist(evtdf.slc.tmatch.eff[is_higgs], histtype="step", density=True)
_ = plt.hist(evtdf.slc.tmatch.eff[is_nu], histtype="step", density=True)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)

var = evtdf.third_trk_dist.copy()
var[np.isnan(var)] = -10

bins = [-12, -8] + list(np.linspace(0, 100, 26))
# bins = np.linspace(0, 100, 26)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.xlabel("Third Track Distance to Vertex [cm]")

plt.ylabel("Area Normalzied")
plt.yscale("log")

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

plt.title("No Cut", color="r")

if dosave: plt.savefig(savedir + "third_track_dist_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)

var = evtdf.max_shw_len.copy()
var[np.isnan(var) | (var < 0)] = -10

bins = [-12, -8] + list(np.linspace(0, 100, 26))

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.xlabel("Max Shower Length [cm]")

plt.ylabel("Area Normalized")
plt.yscale("log")

plt.axvline([20], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()

if dosave: plt.savefig(savedir + "max_shw_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)
var = evtdf.slc.fmatch.score

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Flash Score")

plt.ylabel("Area Normalized")
plt.axvline([20], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "f_score_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)
var = evtdf.slc.nu_score

bins = np.linspace(0, 1, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend(loc="upper left")

plt.xlabel("Pandora $\\nu$-Score")

plt.ylabel("Area Normalized")
plt.axvline([0.4], linestyle=":", color="r", linewidth=2)

plt.text(0.01, 0.4, label, transform=plt.gca().transAxes, fontsize=18)
plt.tight_layout()

if dosave: plt.savefig(savedir + "nu_score_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)
var = evtdf.trunk.chi2pid.I2.chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([40], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_muon_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)
var = evtdf.trunk.chi2pid.I2.chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)
var = evtdf.trunk.chi2pid.I2.chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.end)
var = evtdf.trunk.len

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Len [cm] (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
#nplt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_cont_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & ~TrkInFV(evtdf.trunk.end)
var = evtdf.trunk.len

bins = np.linspace(0, 600, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Len [cm] (Exiting)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_cont_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.end)
var = evtdf.branch.chi2pid.I2.chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([40], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "branch_chi2_muon_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.end)
var = evtdf.branch.chi2pid.I2.chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "branch_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
def is_muon(trk):
    return (TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon < 40) & (trk.chi2pid.I2.chi2_proton > 80)) | (~TrkInFV(trk.end) & (trk.len > 100.))

def is_proton(trk):
    return TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon > 40) & (trk.chi2pid.I2.chi2_proton < 60)

In [ ]:
muselect = is_muon(evtdf.trunk) & is_muon(evtdf.branch)
muselect.name = "2 Muon Tracks Reconstructed"

In [ ]:
branch2trunk = magdf(evtdf.trunk.start - evtdf.branch.start)

when = SlcInFV(evtdf.slc.vertex) & (branch2trunk < 5) #& topselect & muselect & (branch2trunk > 5)
var = evtdf.trunk.med_start_dqdx

bins = np.linspace(0, 2500, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([1000], linestyle=":", color="r", linewidth=2)

plt.text(0.01, 0.8, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")

plt.xlabel("Trunk dQ/dx [ADDC/cm]")
plt.ticklabel_format(axis='y', style='sci', scilimits=(-1,-4))

plt.tight_layout()

if dosave: plt.savefig(savedir + "trunk_med_dqdx_M%i.pdf" % HMASS)

In [ ]:
branch2trunk = magdf(evtdf.trunk.start - evtdf.branch.start)

when = SlcInFV(evtdf.slc.vertex) & (branch2trunk > 5) #& topselect & muselect & (branch2trunk > 5)
var = evtdf.trunk.med_start_dqdx

bins = np.linspace(0, 2500, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([1000], linestyle=":", color="r", linewidth=2)

plt.text(0.01, 0.8, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")

plt.xlabel("Trunk dQ/dx [ADDC/cm]")
plt.ticklabel_format(axis='y', style='sci', scilimits=(-1,-4))

plt.tight_layout()

if dosave: plt.savefig(savedir + "trunk_med_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = np.arccos(dot(evtdf.trunk.dir, evtdf.branch.dir)) #* 180 / np.pi
when = SlcInFV(evtdf.slc.vertex) #& topselect & muselect

bins = np.linspace(0, np.pi, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend(loc="upper right")

plt.axvline([0.6], linestyle=":", color="r", linewidth=2)

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.xlabel("Reconstructed Opening Angle")

if dosave: plt.savefig(savedir + "openangle_M%i.pdf" % HMASS)

In [ ]:
scalar_dir = unit(evtdf.trunk.dir.multiply(recop(evtdf.trunk), axis=0) + 
           evtdf.branch.dir.multiply(recop(evtdf.branch), axis=0))
BEAMDIR = [0.389019125, -0.0572211871, 0.919450845]

beamangle = np.arccos(scalar_dir.x*BEAMDIR[0] + scalar_dir.y*BEAMDIR[1] + scalar_dir.z*BEAMDIR[2])

In [ ]:
var = beamangle

when = SlcInFV(evtdf.slc.vertex) & muselect

bins = np.linspace(0, np.pi, 32)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([0.3], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Reconstructed Scalar Angle to Beam [rad]")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_beam_angle_relativenorm_M%i.pdf" % HMASS)

In [ ]:
var = np.abs((recop(evtdf.trunk) - recop(evtdf.branch)) / (recop(evtdf.trunk) + recop(evtdf.branch)))

when = SlcInFV(evtdf.slc.vertex) & muselect

bins = np.linspace(0, 1, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend(loc="lower left")
# plt.axvline([0.3], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Momentum Assymetry")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.4, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "momentum_balance_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l0_5cm.dqdx

when = SlcInFV(evtdf.slc.vertex) & muselect # & (~is_higgs | ((evtdf.trunk.truth.p.genE > 0.5) & (evtdf.branch.truth.p.genE > 0.5)))

bins = np.linspace(0, 8e5, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([5e5], linestyle=":", color="r", linewidth=2)

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Stub dQ/dx [#elec/cm] (len < 0.5cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_0_5cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l1cm.dqdx

bins = np.linspace(0, 8e5, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([3e5], linestyle=":", color="r", linewidth=2)

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Stub dQ/dx [#elec/cm] (0.5cm < len < 1cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_0_5cm_1cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l2cm.dqdx

bins = np.linspace(0, 8e5, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([2.5e5], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Stub dQ/dx [#elec/cm] (1cm < len < 2cm)")
plt.title("ICARUS MC")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)
plt.ylabel("Area Normalized")

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_1cm_2cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l3cm.dqdx

bins = np.linspace(0, 8e5, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([2.5e5], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Stub dQ/dx [#elec/cm] (2cm < len < 3cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_2cm_3cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
hasstub = (evtdf.stub.l0_5cm.dqdx > 5e5) | (evtdf.stub.l1cm.dqdx > 3e5) | (evtdf.stub.l2cm.dqdx > 2.5e5) | (evtdf.stub.l2cm.dqdx > 1.5e5)

In [ ]:
preselect = [
    SlcInFV(evtdf.slc.vertex),
    TrkInFV(evtdf.trunk.end) == TrkInFV(evtdf.branch.end)
]

objects = [
    np.isnan(evtdf.max_shw_len) #| (evtdf.max_shw_len < 20),
]

scores = [
    # evtdf.slc.fmatch.score < 20,
    evtdf.slc.nu_score > 0.4,
]

muid = [
    evtdf.trunk.len > 25,
    evtdf.branch.len > 25,
    ((~TrkInFV(evtdf.trunk.end) & (evtdf.trunk.len > 50)) |\
     (TrkInFV(evtdf.trunk.end) & (evtdf.trunk.chi2pid.I2.chi2_muon < 20) & (evtdf.trunk.chi2pid.I2.chi2_proton > 60))),
    ((~TrkInFV(evtdf.branch.end) & (evtdf.branch.len > 50)) |\
     (TrkInFV(evtdf.branch.end) & (evtdf.branch.chi2pid.I2.chi2_muon < 20) & (evtdf.branch.chi2pid.I2.chi2_proton > 60))),
    (evtdf.trunk.med_start_dqdx > 1100) | np.isnan(evtdf.trunk.med_start_dqdx) | (magdf(evtdf.trunk.start - evtdf.branch.start) < 5),    
]

kinematics = [
    np.arccos(dot(evtdf.trunk.dir, evtdf.branch.dir)) < 0.5,
    beamangle < 0.2,
]

stubs = [
    ~hasstub,
]

def cut_all(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 & c
    return l0

# cuts = preselect + objects + scores + muid + kinematics + stubs
cuts = preselect + objects + scores + muid + stubs + kinematics

passcut = cut_all(cuts)

In [ ]:
var = evtdf.trunk.trkd.len
var[np.isnan(var) | (var < 0)] = -10

bins = [-12, -8] + list(np.linspace(0, 100, 26))

for c in categories:
    _ = plt.hist(var[when & c & passcut], weights=scale[when & c & passcut], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()
plt.yscale("log")

plt.tight_layout()
plt.xlabel("(Trunk) Longest Daughter Track")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "trunk_longest_daughter_track_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.branch.trkd.len
var[np.isnan(var) | (var < 0)] = -10

bins = [-12, -8] + list(np.linspace(0, 100, 26))

for c in categories:
    _ = plt.hist(var[when & c & passcut], weights=scale[when & c & passcut], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()
plt.yscale("log")

plt.tight_layout()
plt.xlabel("(Branch) Longest Daughter Track")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "branch_longest_daughter_track_M%i.pdf" % HMASS)

In [ ]:
var = dot(evtdf.trunk.dir, evtdf.trunk.trkd.dir)
var[np.isnan(var)] = -1.1

bins = [-1.1 - 0.025, -1.1 + 0.025] + list(np.linspace(-1, 1, 21))

for c in categories:
    _ = plt.hist(var[when & c  & passcut], weights=scale[when & c & passcut], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()
plt.yscale("log")

plt.tight_layout()
plt.xlabel("(Trunk) Longest Daughter Track Cos-Angle")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "trunk_longest_daughter_track_costh_M%i.pdf" % HMASS)

In [ ]:
var = dot(evtdf.branch.dir, evtdf.branch.trkd.dir)
var[np.isnan(var)] = -1.1

bins = [-1.1 - 0.025, -1.1 + 0.025] + list(np.linspace(-1, 1, 21))

for c in categories:
    _ = plt.hist(var[when & c & passcut], weights=scale[when & c & passcut], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()
plt.yscale("log")

plt.tight_layout()
plt.xlabel("(Branch) Longest Daughter Track Cos-Angle")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "branch_longest_daughter_track_costh_M%i.pdf" % HMASS)

In [ ]:
piveto = [
    np.isnan(evtdf.trunk.trkd.len) | (evtdf.trunk.trkd.len < 25),
    np.isnan(evtdf.branch.trkd.len) | (evtdf.branch.trkd.len < 15)
]

cuts = preselect + objects + scores + muid + kinematics + stubs + piveto


cutlist = [preselect + objects, scores, muid, stubs, kinematics]
cutlist_names = ["Pre-Select", "Score Cuts", "2$\\mu$ ID", "Stubs", "Kinematics", ]

passcut = cut_all(cuts)

In [ ]:
MUON_MASS = 0.1057
Etrunk = np.sqrt(MUON_MASS**2 + recop(evtdf.trunk)**2)
Ebranch = np.sqrt(MUON_MASS**2 + recop(evtdf.branch)**2)

s_mass = np.sqrt(2)*np.sqrt(MUON_MASS**2 + Etrunk*Ebranch - recop(evtdf.trunk)*recop(evtdf.branch)*dot(evtdf.branch.dir, evtdf.trunk.dir))

In [ ]:
bdt = xgb.XGBRegressor(learning_rate=0.1)

In [ ]:
xdata = [
    evtdf.max_shw_len,
    evtdf.slc.fmatch.score,
    evtdf.slc.nu_score,
    evtdf.trunk.len,
    evtdf.branch.len,
    TrkInFV(evtdf.trunk.end),
    evtdf.trunk.chi2pid.I2.chi2_muon,
    evtdf.trunk.chi2pid.I2.chi2_proton,
    TrkInFV(evtdf.branch.end),
    evtdf.branch.chi2pid.I2.chi2_muon,
    evtdf.branch.chi2pid.I2.chi2_proton,
    evtdf.trunk.med_start_dqdx,
    np.arccos(dot(evtdf.trunk.dir, evtdf.branch.dir)),
    beamangle,
    evtdf.stub.l0_5cm.dqdx,
    evtdf.stub.l1cm.dqdx,
    evtdf.stub.l2cm.dqdx,
    evtdf.stub.l3cm.dqdx,
    evtdf.trunk.trkd.len,
    evtdf.branch.trkd.len,
    s_mass
]

labels = [
    "Max Shw Len",
    "Flash Match Score",
    "Nu Score",
    "Trunk Length",
    "Branch Length",
    "Trunk Contained",
    "Trunk chi2 mu",
    "Trunk chi2 p",
    "Branch Contained",
    "Branch chi2 mu",
    "Branch chi2 p",
    "Trunk Start dQ/dx",
    "Open Angle",
    "Beam Angle",
    "Stub dQ/dx 0_5cm",
    "Stub dQ/dx 1cm",
    "Stub dQ/dx 2cm",
    "Stub dQ/dx 3cm",
    "Trunk Daughter Length",
    "Branch Daughter Length",
    "Scalar Mass",
]

xdata = np.vstack(xdata).T
ydata = is_higgs

when = SlcInFV(evtdf.slc.vertex)

In [ ]:
ddf = pd.DataFrame(xdata, columns=labels, index=evtdf.index)

In [ ]:
cutdf = pd.DataFrame(np.stack([cut_all(c).values for c in cutlist]).T, columns=cutlist_names, index=evtdf.index)

In [ ]:
scale = evtdf.scale.copy()
# scale[is_higgs] = evtdf.scale*1e3

In [ ]:
plt.figure(figsize=(9, 6))

bins = np.linspace(0, 10, 21)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & (evtdf.slc.tmatch.eff > 0.5)
# slc_is_select = slc_is_reco & passcut

thiscut = True
for c, name in zip(cutlist, cutlist_names):
    thiscut = cut_all(c) & thiscut
    when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
    D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.65, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % HMASS)

In [ ]:
plt.figure(figsize=(9, 6))

bins = np.linspace(0, 10, 21)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & TrkInFV(evtdf.trunk.end) & TrkInFV(evtdf.branch.end)
# slc_is_select = slc_is_reco & passcut

thiscut = True
for c, name in zip(cutlist, cutlist_names):
    thiscut = cut_all(c) & thiscut
    when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
    D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.65, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % HMASS)

In [ ]:
h_mcdf

In [ ]:
evtdf.slc.tmatch[is_nu]

In [ ]:
evtdf.slc.truth[is_higgs]

In [ ]:
(evtdf.slc.tmatch.idx == 0)[is_higgs].sum()

In [ ]:
((evtdf.slc.tmatch.idx == 0) & (evtdf.slc.tmatch.eff > 0.5))[is_higgs].groupby(level=[0,1]).any().sum()

In [ ]:
bins = np.concatenate((np.linspace(0, 4, 16), [5, 6, 8, 10]))

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = cut_all(preselect) & (evtdf.slc.tmatch.idx == 0) & (evtdf.slc.tmatch.eff > 0.5) 
slc_is_select = slc_is_reco & passcut

when_reco = when_all & slc_is_reco[is_higgs].groupby(level=[0, 1]).any()
when_select = when_all & slc_is_select[is_higgs].groupby(level=[0, 1]).any()

D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
Nreco, _ = np.histogram(h_mcdf[when_reco].E, bins=bins)
Nslct, _ = np.histogram(h_mcdf[when_select].E, bins=bins)

centers = (bins[1:] + bins[:-1])/2.

plt.plot(centers, Nreco/D, label="Pre-Selection")
plt.plot(centers, Nslct/D, label="Full Selection")
plt.legend(loc="upper right")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
# plt.text(0.25, 0.05, label, transform=plt.gca().transAxes, fontsize=18)
plt.text(0.6, 0.5, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_M%i.pdf" % HMASS)

In [ ]:
bins

In [ ]:
Nslct/D

In [ ]:
width = 0.25
plt.figure(figsize=(6.4, 6.4))
for i,category in enumerate(categories):
    plt.barh(np.arange(len(cutlist)) + i*width, 
             [np.sum(evtdf.scale[category & cut_all([c for cuts in cutlist[:i+1] for c in cuts])]) for i in range(len(cutlist))], 
             width, 
             label=category.name,
             tick_label = cutlist_names
            )
    plt.xscale("log")
    plt.legend(ncol=3, bbox_to_anchor=(-0.05, 1.0, 1, 0.2), loc="upper left")
    plt.xlabel("Scalar Candidates / %s (1yr)" % POTSTR)
plt.xlim([1, plt.xlim()[1]])
plt.title(label.replace("\n", ", "))
plt.tight_layout()
if dosave: plt.savefig(savedir + "cutchart_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.trunk.len

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 500, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

plt.xlabel("Trunk Length [cm]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.text(0.25, 0.6, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_trunk_length_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.branch.len

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 500, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

plt.tight_layout()
plt.xlabel("Branch Length [cm]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.text(0.25, 0.6, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_branch_length_M%i.pdf" % HMASS)

In [ ]:
var = recop(evtdf.trunk)

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 3, 31)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

plt.tight_layout()
plt.xlabel("Trunk Momentum [GeV/c]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.text(0.6, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_trunk_momentum_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.slc.truth.max_proton_ke

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 0.1, 51)

for c in [is_nu]: 
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2) 
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4)) plt.tight_layout() plt.xlabel("Max Proton KE [GeV]") plt.title("ICARUS MC") plt.ylabel("Candidates / 6e20 POT")

plt.tight_layout() 
if dosave: plt.savefig(savedir + "passcut_proton_ke.pdf")


In [ ]:
evtdf.slc.truth.max_proton_ke[SlcInFV(evtdf.slc.vertex) & passcut & is_nu].max()

In [ ]:
scale[SlcInFV(evtdf.slc.vertex) & passcut & (TrkInFV(evtdf.trunk.end) != TrkInFV(evtdf.branch.end)) & is_higgs].sum()

In [ ]:
evtdf.slc.truth.np

In [ ]:
var = s_mass

when = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.truth.np < 1) & passcut

bins = np.linspace(0.2, 0.6, 21)

names = [
    categories[0].name,
    "$\\nu$ no p",
    categories[2].name
]

for c,n in zip(categories, names):
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=n, 
                 histtype="step", linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_nop_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & passcut

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("$M_{\\mu\\mu}$ [GeV/c$^2$]")
plt.ylabel("Events / %s (1yr)" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & passcut & TrkInFV(evtdf.trunk.end) & TrkInFV(evtdf.branch.end)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_M%i_contained.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & passcut & (TrkInFV(evtdf.trunk.end) != TrkInFV(evtdf.branch.end))

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_M%i_semicontained.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & passcut & ~TrkInFV(evtdf.trunk.end) & ~TrkInFV(evtdf.branch.end)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_M%i_exiting.pdf" % HMASS)

In [ ]:
even = evtdf.index.get_level_values(0) % 2 == 0

train = even
test = ~even

In [ ]:
# bdt = bdt.fit(xdata[when & train], ydata[when & train], sample_weight=evtdf.scale[when & train])
# bdt.get_booster().feature_names = labels

In [ ]:
# score = bdt.predict(xdata)

In [ ]:
muid = [
    evtdf.trunk.len > 25,
    evtdf.branch.len > 25,
    ((~TrkInFV(evtdf.trunk.end) & (evtdf.trunk.len > 50)) |\
     (TrkInFV(evtdf.trunk.end) & (evtdf.trunk.chi2pid.I2.chi2_muon < 20) & (evtdf.trunk.chi2pid.I2.chi2_proton > 60))),
    ((~TrkInFV(evtdf.branch.end) & (evtdf.branch.len > 50)) |\
     (TrkInFV(evtdf.branch.end) & (evtdf.branch.chi2pid.I2.chi2_muon < 20) & (evtdf.branch.chi2pid.I2.chi2_proton > 60))),
    (evtdf.trunk.med_start_dqdx > 1100) | np.isnan(evtdf.trunk.med_start_dqdx) | (magdf(evtdf.trunk.start - evtdf.branch.start) < 5),    
]

In [ ]:
SlcInFV(nu_mcdf.position).sum()

In [ ]:
(1 - evtdf.scale[(SlcInFV(evtdf.slc.truth.position) & is_nu & SlcInFV(evtdf.slc.vertex) & passcut)].sum() / nu_mcdf.scale[SlcInFV(nu_mcdf.position)].sum())*100

In [ ]:
modes = [0, 1, 2, 3, 10]
names = ["QE", "RES", "DIS", "COH", "MEC"]

for m, n in zip(modes, names):
    frac = nu_mcdf.scale[(SlcInFV(nu_mcdf.position) & (nu_mcdf.genie_mode == m))].sum() / nu_mcdf.scale[SlcInFV(nu_mcdf.position)].sum()
    print("%s: %.1f%%" % (n, frac*100))

In [ ]:
modes = [0, 10, 1, 2, 3]
names = ["QE", "MEC","RES", "DIS", "COH"]

for m, n in zip(modes, names):
    frac = evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut & (evtdf.slc.truth.genie_mode == m))].sum() / evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut)].sum()
    print("%s: %.1f%%" % (n, frac*100))

In [ ]:
nu_mcdf.scale[(SlcInFV(nu_mcdf.position) & (nu_mcdf.pdg == -14))].sum() / nu_mcdf.scale[SlcInFV(nu_mcdf.position)].sum()

In [ ]:
evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut & (evtdf.slc.truth.pdg == -14))].sum() / evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut)].sum()

In [ ]:
nu_mcdf.scale[(SlcInFV(nu_mcdf.position) & (nu_mcdf.np == 0))].sum() / nu_mcdf.scale[SlcInFV(nu_mcdf.position)].sum()

In [ ]:
evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut & (evtdf.slc.truth.np == 0))].sum() / evtdf.scale[(SlcInFV(evtdf.slc.vertex) & is_nu & passcut)].sum()

In [ ]:
var = evtdf.slc.truth.E

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 15, 31)

# for c in nucategories:
_ = plt.hist([var[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], 
             weights=[scale[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_nue_intmodes.pdf")

In [ ]:
var = evtdf.slc.truth.lmom

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 7, 8)

# for c in nucategories:
_ = plt.hist([var[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], 
             weights=[scale[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend(ncol=1)

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Muon Momentum [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_lmom_intmodes.pdf")

In [ ]:
var = evtdf.slc.truth.E

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 15, 31)

_ = plt.hist([var[when & c] for c in nucategories], 
             weights=[scale[when & c] for c in nucategories], bins=bins, 
             label=[c.name for c in nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_nue_categories.pdf")

In [ ]:
var = evtdf.slc.truth.lmom

when = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 10, 21)

_ = plt.hist([var[when & c] for c in nucategories], 
             weights=[scale[when & c] for c in nucategories], bins=bins, 
             label=[c.name for c in nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_lmom_categories.pdf")

In [ ]:
(SlcInFV(nu_mcdf.position) & (nu_mcdf.max_proton_ke < 0.0001)).sum() / SlcInFV(nu_mcdf.position).sum()

In [ ]:
(when & is_nu & (evtdf.slc.truth.max_proton_ke < 0.005)).sum(), (when & is_nu & (evtdf.slc.truth.max_proton_ke > 0.005)).sum()

In [ ]:
np.sum(scale[when & is_nu & (evtdf.slc.truth.max_proton_ke > 0.005)]), np.sum(scale[when & is_nu & (evtdf.slc.truth.max_proton_ke < 0.005)])

In [ ]:
when = SlcInFV(evtdf.slc.vertex) &  evtdf.third_trk_contained
var = evtdf.third_trk_chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "thirdtrk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & evtdf.third_trk_contained
var = evtdf.third_trk_chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([40], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "thirdtrk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
trunk_dist = magdf(evtdf.trunk.start - evtdf.slc.vertex)
branch_dist = magdf(evtdf.branch.start - evtdf.slc.vertex)

In [ ]:
third_trk_is_proton = (evtdf.third_trk_chi2_muon > 40) & (evtdf.third_trk_chi2_proton < 60) & evtdf.third_trk_contained

sidebandcut = (third_trk_is_proton | is_proton(evtdf.trunk) | is_proton(evtdf.branch)) &\
    (trunk_dist < 10) & (branch_dist < 10) & (evtdf.third_trk_dist < 10)

# sidebandcut = third_trk_is_proton & (trunk_dist < 10) & (branch_dist < 10) & (evtdf.third_trk_dist < 10)

In [ ]:

# plt.figure(figsize=(9, 6))

bins = np.linspace(0, 3, 31)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & (evtdf.slc.tmatch.eff > 0.5)
# slc_is_select = slc_is_reco & passcut

thiscut = sidebandcut
when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  


centers = (bins[1:] + bins[:-1])/2.

plt.plot(centers, N/D, label="Scalar Eff")
plt.legend()

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.6, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
plt.ylim([0, 0.1])
if dosave: plt.savefig(savedir + "scalar_eff_3trk_sideband_M%i.pdf" % HMASS)

In [ ]:
# plt.figure(figsize=(9, 6))

bins = np.linspace(0, 5, 31)

when_all = SlcInFV(nu_mcdf.position)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx >= 0) & (evtdf.slc.tmatch.eff > 0.5) &\
    SlcInFV(evtdf.slc.truth.position)
# slc_is_select = slc_is_reco & passcut

when_cut = slc_is_reco & sidebandcut
# when_cut = when_all & (slc_is_reco & thiscut)[is_nu].groupby(level=0).any()
D, _ = np.histogram(nu_mcdf[when_all].E, bins=bins)
N, _ = np.histogram(evtdf.slc.truth.E[when_cut], bins=bins)  


centers = (bins[1:] + bins[:-1])/2.

plt.plot(centers, N/D, label="Neutrino Eff")
# plt.legend()

plt.xlabel("Neutrino Energy [GeV]")
plt.ylabel("Efficiency")

plt.tight_layout()
# plt.ylim([0, 0.1])
plt.text(0.6, 0.05, "Total Eff: %.1f%%" % (100*np.sum(N) / np.sum(D)), transform=plt.gca().transAxes, fontsize=18)
if dosave: plt.savefig(savedir + "nu_eff_3trk_sideband.pdf")

In [ ]:
var = evtdf.slc.truth.E

bins = np.linspace(0, 10, 21)


# for c in nucategories:
_ = plt.hist([var[when_cut & (evtdf.slc.truth.genie_mode == m) & evt_numu_cc] for m in modes], 
             weights=[scale[when_cut & (evtdf.slc.truth.genie_mode == m) & evt_numu_cc] for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "pass_3trk_sideband_nue.pdf")

In [ ]:
var = evtdf.slc.truth.lmom

bins = np.linspace(0, 6, 19)

# for c in nucategories:
_ = plt.hist([var[when_cut & (evtdf.slc.truth.genie_mode == m) & evt_numu_cc] for m in modes], 
             weights=[scale[when_cut & (evtdf.slc.truth.genie_mode == m) & evt_numu_cc] for m in modes], bins=bins, 
             label=names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Muon Momentum [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "pass_3trk_sideband_lmom.pdf")

In [ ]:
var = np.maximum(evtdf.trunk.len, evtdf.branch.len)

when = SlcInFV(evtdf.slc.vertex) & ~np.isnan(evtdf.slc.nu_score)

bins = np.linspace(0, 400, 21)

# for c in nucategories:
_ = plt.hist([var[sidebandcut & c] for c in [is_cosmic] + nucategories], 
             weights=[scale[sidebandcut & c] for c in [is_cosmic] + nucategories], bins=bins, 
             label=[c.name for c in [is_cosmic] + nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Maximum Track Length [cm]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "sideband_trklen.pdf")

In [ ]:
var = evtdf.slc.truth.E

bins = np.linspace(0, 10, 21)

# for c in nucategories:
_ = plt.hist([var[when_cut & c] for c in nucategories], 
             weights=[scale[when_cut & c] for c in nucategories], bins=bins, 
             label=[c.name for c in nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "pass_3trk_sideband_nue_categories.pdf")

In [ ]:
var = evtdf.slc.truth.lmom

bins = np.linspace(0, 6, 19)

# for c in nucategories:
_ = plt.hist([var[when_cut & c] for c in nucategories], 
             weights=[scale[when_cut & c] for c in nucategories], bins=bins, 
             label=[c.name for c in nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Muon Momentum [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "pass_3trk_sideband_lmom_categories.pdf")